In [14]:
import os
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 

In [15]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]]) 

OneHotEncoder()

In [16]:
# This cell updates result list for images with tumor

data = []
paths = []
result = []

for r, d, f in os.walk(r'C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\pneumonia'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())

In [17]:
paths = []
for r, d, f in os.walk(r"C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\normal"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[1]]).toarray())

In [27]:
data = np.array(data)
data.shape

(538, 128, 128, 3)

In [28]:
result = np.array(result)
result = result.reshape(538,1)

ValueError: cannot reshape array of size 1076 into shape (538,1)

In [20]:
x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='same'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Changed to 1 neuron and 'sigmoid' activation for binary classification

model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 128, 128, 32)      416       
                                                                 
 conv2d_9 (Conv2D)           (None, 128, 128, 32)      4128      
                                                                 
 batch_normalization_4 (Bat  (None, 128, 128, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 64, 64, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 64, 64, 64)       

In [22]:
y_train.shape

(430, 2)

In [23]:
history = model.fit(x_train, y_train, epochs = 30, batch_size = 40, verbose = 1,validation_data = (x_test, y_test))

Epoch 1/30


ValueError: in user code:

    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [29]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [30]:
# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Prepare data and labels
data = []
result = []

# Read pneumonia images
for r, d, f in os.walk(r'C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\pneumonia'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(0)  # Label 0 for pneumonia

# Read normal images
for r, d, f in os.walk(r"C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\normal"):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(1)  # Label 1 for normal

In [31]:
# Convert to numpy arrays
data = np.array(data)
result = np.array(result)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

# Build the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='same'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

In [33]:
# Train the model
history = model.fit(x_train, y_train, epochs=30, batch_size=40, verbose=1, validation_data=(x_test, y_test))

Epoch 1/30
11/11 [==============================] - 13s 1s/step - loss: 14.1303 - accuracy: 0.6814 - val_loss: 25.6792 - val_accuracy: 0.8056
Epoch 2/30
11/11 [==============================] - 12s 1s/step - loss: 5.0445 - accuracy: 0.7233 - val_loss: 13.0667 - val_accuracy: 0.8056
Epoch 3/30
11/11 [==============================] - 12s 1s/step - loss: 2.3789 - accuracy: 0.7651 - val_loss: 5.5333 - val_accuracy: 0.8056
Epoch 4/30
11/11 [==============================] - 12s 1s/step - loss: 1.3785 - accuracy: 0.7512 - val_loss: 2.3809 - val_accuracy: 0.8056
Epoch 5/30
11/11 [==============================] - 12s 1s/step - loss: 0.6669 - accuracy: 0.8349 - val_loss: 0.7910 - val_accuracy: 0.7778
Epoch 6/30
11/11 [==============================] - 12s 1s/step - loss: 0.3894 - accuracy: 0.8302 - val_loss: 0.6963 - val_accuracy: 0.8056
Epoch 7/30
11/11 [==============================] - 13s 1s/step - loss: 0.3058 - accuracy: 0.8628 - val_loss: 0.4689 - val_accuracy: 0.7778
Epoch 8/30
11/11 

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Set seed for reproducibility
np.random.seed(8)

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Prepare data and labels
data = []
result = []

# Read pneumonia images
for r, d, f in os.walk(r'C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\pneumonia'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(0)  # Label 0 for pneumonia

# Read normal images
for r, d, f in os.walk(r"C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\normal"):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(1)  # Label 1 for normal

# Convert to numpy arrays
data = np.array(data)
result = np.array(result)

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

# Build the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='same'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Changed to 1 neuron and 'sigmoid' activation for binary classification

model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=30, batch_size=40, verbose=1, validation_data=(x_test, y_test))

# Print model summary
print(model.summary())


In [34]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [35]:
# Set seed for reproducibility
np.random.seed(8)

In [36]:
# Prepare data and labels
data = []
result = []

# Read pneumonia images
for r, d, f in os.walk(r'C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\pneumonia'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(0)  # Label 0 for pneumonia

# Read normal images
for r, d, f in os.walk(r"C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\normal"):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(1)  # Label 1 for normal

In [37]:
# Convert to numpy arrays
data = np.array(data)
result = np.array(result)

# Initialize K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=8)

# Initialize lists to store evaluation results
train_accs = []
val_accs = []

# Iterate over folds
for train_index, val_index in kfold.split(data):
    x_train, x_val = data[train_index], data[val_index]
    y_train, y_val = result[train_index], result[val_index]

In [38]:
# Build the model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='same'))
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

IndentationError: unexpected indent (1141349083.py, line 2)

In [43]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Set seed for reproducibility
np.random.seed(8)

# Prepare data and labels
data = []
result = []

# Read pneumonia images
for r, d, f in os.walk(r'C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\pneumonia'):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(0)  # Label 0 for pneumonia

# Read normal images
for r, d, f in os.walk(r"C:\Users\pirat\Downloads\pneumonia 2-20240125T203918Z-001\classification\normal"):
    for file in f:
        if '.jpg' in file:
            img = Image.open(os.path.join(r, file))
            img = img.resize((128, 128))
            img = np.array(img)
            if img.shape == (128, 128, 3):
                data.append(np.array(img))
                result.append(1)  # Label 1 for normal

# Convert to numpy arrays
data = np.array(data)
result = np.array(result)

# Initialize K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=8)

# Initialize lists to store evaluation results
train_accs = []
val_accs = []

# Iterate over folds
for train_index, val_index in kfold.split(data):
    x_train, x_val = data[train_index], data[val_index]
    y_train, y_val = result[train_index], result[val_index]
    model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

    # Train the model
    history = model.fit(x_train, y_train, epochs=30, batch_size=40, verbose=1, validation_data=(x_val, y_val))

    # Evaluate the model on training data
    train_loss, train_acc = model.evaluate(x_train, y_train, verbose=0)
    train_accs.append(train_acc)

    # Evaluate the model on validation data
    val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
    val_accs.append(val_acc)

# Calculate and print mean accuracy
print("Mean training accuracy:", np.mean(train_accs))
print("Mean validation accuracy:", np.mean(val_accs))


Epoch 1/30
11/11 [==============================] - 13s 1s/step - loss: 7.7896 - accuracy: 0.7000 - val_loss: 0.5546 - val_accuracy: 0.7870
Epoch 2/30
11/11 [==============================] - 12s 1s/step - loss: 0.5609 - accuracy: 0.7512 - val_loss: 0.5340 - val_accuracy: 0.7870
Epoch 3/30
11/11 [==============================] - 13s 1s/step - loss: 0.5463 - accuracy: 0.7837 - val_loss: 0.5067 - val_accuracy: 0.7870
Epoch 4/30
11/11 [==============================] - 13s 1s/step - loss: 0.5093 - accuracy: 0.7814 - val_loss: 0.4765 - val_accuracy: 0.7870
Epoch 5/30
11/11 [==============================] - 13s 1s/step - loss: 0.4877 - accuracy: 0.7884 - val_loss: 0.4634 - val_accuracy: 0.7870
Epoch 6/30
 3/11 [=======>......................] - ETA: 8s - loss: 0.4358 - accuracy: 0.7917

KeyboardInterrupt: 